In [48]:
import os
import re
import cupy as cp
import numpy as np
import pandas as pd
import nibabel as nib
import pydicom
from datetime import datetime

In [58]:
datas_path = r'E:\yby\python\Breast_Type19\dataset\datas_path.csv'
paths=pd.read_csv(datas_path)
# paths.head()

In [59]:
# for i in range(paths.shape[0]):
i=200
label=nib.load(paths.loc[i,'label_path']).get_fdata()
label_gpu = cp.array(label)
print(label_gpu.shape)
locs=label.nonzero()
# print(locs)
rows=locs[0]
# print(rows)
columns=locs[1]
slices=locs[2]

(512, 512, 116)


In [60]:
stages = [int(stage) for stage in os.listdir(paths.loc[i,'nii_data']) if not re.search('nii',stage)]
stages.sort()
stages_path=[os.path.join(paths.loc[i,'nii_data'],str(stage)) for stage in stages]
dcm_paths=[os.path.join(paths.loc[i,'dcm_data'],str(stage)) for stage in stages]
if paths.loc[i,'grade']== 0:
    grade='Benign'
else:
    grade='Malignant'
csv_name = grade+'_'+paths.loc[i,'patient_name']+'_'+paths.loc[i,'label_name']+'.csv'
print(csv_name)
csv_columns=[str(stage) for stage in stages]
[csv_columns.append(name) for name in ['rows','columns','slices']]
csv=pd.DataFrame(columns=csv_columns)
data=[]
set_columns=['filp_angle','TR']
set=[]

for i in range(len(stages)):
    stage_path=os.path.join(stages_path[i],os.listdir(stages_path[i])[0])
    # print(stages[i],' path: ',stage_path)
    dcms=os.path.join(dcm_paths[i],os.listdir(dcm_paths[i])[0])
    # print(stages[i],' path: ',dcms)
    ds = pydicom.read_file(dcms)

    if i==0:
        #读取固定的属性
        # filp angle
        filp_angle=ds[0X0018,0X1314].value
        # print(filp_angle)
        set.append(filp_angle
                   )
        TR=ds[0X0018,0X0080].value
        # print(TR)
        set.append(TR)

    str_time=ds[0X0008,0X0032].value
    print(str_time)
    # time_various='time_'+str(i+1)+'=time'
    set_columns.append('time_'+str(i+1))
    time=datetime.strftime(datetime.strptime(str_time,"%H%M%S"), "%H:%M:%S")
    # print(time_various)
    #扫描时间
    # exec(time_various)
    set.append(time)
    stage_data = nib.load(stage_path).get_fdata()
    stage_data_gpu = cp.array(stage_data)
    label2data=cp.multiply(label_gpu,stage_data_gpu)
    data.append(label2data)
    # for j in range(len(rows)):
    #     temp = stage_data_gpu[rows[j],columns[j],slices[j]]
    #     csv.loc[j,str(i)]= temp
    #     if i==0:
    #         csv.loc[j,['rows','columns','slices']]=[rows[j],columns[j],slices[j]]
        # print(temp)


Malignant_P070_NOR_87.nii.csv
174842


D:\anaconda\envs\Type19\lib\site-packages\pydicom\charset.py:715: UserWarning: Value 'GB18030' cannot be used as code extension, ignoring it
  encodings, py_encodings


175103
175140
175217
175254
175331
175408
175445
175522
175559
175636


In [53]:
set_csv=np.array([set])
# print(set_csv)
pd_set=pd.DataFrame(set_csv,columns=set_columns)
pd_set

,filp_angle,TR,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,time_10
0,12,4.224,14:11:14,14:12:16,14:13:02,14:13:47,14:14:32,14:15:18,14:16:03,14:16:49,14:17:34,14:18:19


In [5]:
cp_data=cp.array(data)
cp_data=cp_data[cp_data!=0]

In [6]:
test=cp.reshape(cp_data,(-1,152))

In [7]:
print(test.T)

[[1003. 1285. 1462. ... 1527. 1527. 1626.]
 [1069. 1279. 1437. ... 1845. 2013. 2008.]
 [ 876.  887.  966. ... 1140. 1263. 1310.]
 ...
 [ 967. 1327. 1627. ... 2651. 2800. 2903.]
 [1060. 1282. 1494. ... 2474. 2697. 2729.]
 [1033. 1218. 1395. ... 1875. 1991. 2111.]]


In [8]:
print(test.T.shape)

(152, 10)


In [9]:
TIC=test.T

In [10]:
result = cp.c_[TIC,rows,columns,slices]

In [11]:
result

array([[1003., 1285., 1462., ...,   72.,  185.,   48.],
       [1069., 1279., 1437., ...,   72.,  186.,   48.],
       [ 876.,  887.,  966., ...,   72.,  186.,   50.],
       ...,
       [ 967., 1327., 1627., ...,   78.,  186.,   47.],
       [1060., 1282., 1494., ...,   78.,  186.,   50.],
       [1033., 1218., 1395., ...,   78.,  187.,   47.]])

In [12]:
csv=pd.DataFrame(result,columns=csv_columns)

In [13]:
csv

,1,2,3,4,5,6,7,8,9,10,rows,columns,slices
0,1003.0,1285.0,1462.0,1576.0,1575.0,1583.0,1565.0,1527.0,1527.0,1626.0,72.0,185.0,48.0
1,1069.0,1279.0,1437.0,1607.0,1691.0,1760.0,1842.0,1845.0,2013.0,2008.0,72.0,186.0,48.0
2,876.0,887.0,966.0,1001.0,989.0,1200.0,1176.0,1140.0,1263.0,1310.0,72.0,186.0,50.0
3,912.0,1109.0,1150.0,1200.0,1179.0,1191.0,1222.0,1242.0,1391.0,1390.0,72.0,187.0,48.0
4,978.0,1191.0,1324.0,1401.0,1374.0,1542.0,1556.0,1472.0,1604.0,1659.0,72.0,187.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,992.0,1365.0,1693.0,1977.0,2182.0,2253.0,2372.0,2395.0,2439.0,2502.0,78.0,185.0,47.0
148,1138.0,1580.0,2115.0,2552.0,2928.0,3154.0,3393.0,3523.0,3627.0,3779.0,78.0,185.0,50.0
149,967.0,1327.0,1627.0,1856.0,2117.0,2369.0,2534.0,2651.0,2800.0,2903.0,78.0,186.0,47.0
150,1060.0,1282.0,1494.0,1647.0,1894.0,2099.0,2370.0,2474.0,2697.0,2729.0,78.0,186.0,50.0
